In [30]:
import pydot
import numpy as np
from IPython.display import display, Javascript, SVG
from pydrake.examples.manipulation_station import ManipulationStation
from manipulation.scenarios import (
    AddIiwa, AddShape
)
from manipulation.meshcat_cpp_utils import (
    StartMeshcat
)
from pydrake.all import (
    AddMultibodyPlantSceneGraph, DiagramBuilder, MeshcatVisualizerCpp, MeshcatVisualizerParams, Parser, 
    RollPitchYaw, RigidTransform, RevoluteJoint, Sphere, Simulator, InverseDynamicsController, MultibodyPlant
)
import pydrake.all

In [31]:
meshcat = StartMeshcat()

In [33]:
def AddFloatingIiwa(plant, collision_model="no_collision"):
    sdf_path = "iiwa_rock_climbing/models/iiwa_description/iiwa7/" + \
        f"iiwa7_{collision_model}.sdf"

    parser = Parser(plant)
    iiwa = parser.AddModelFromFile(sdf_path)
    
    # Set default positions:
    q0 = [0.0, 0.1, 0, -1.2, 0, 1.6, 0]
    index = 0
    for joint_index in plant.GetJointIndices(iiwa):       
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(q0[index])
            index += 1
        
    return iiwa

def AddFloatingBase(plant, iiwa):
    base = AddShape(plant, pydrake.geometry.Box(1,1,1), "mobile_base")
    plant.WeldFrames(plant.GetFrameByName("mobile_base"), plant.GetFrameByName("iiwa_link_0"))
    # TODO: Add offset
    # plant.WeldFrames(
    #     plant.GetFrameByName("iiwa_link_7"), 
    #     plant.GetFrameByName("base_link"), 
    #     RigidTransform(RollPitchYaw(0, -np.pi/2, 0), [0, 0, 0.25]))
    return base

In [43]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)

iiwa = AddFloatingIiwa(plant)
base = AddFloatingBase(plant, iiwa)
parser = Parser(plant)

# ball = parser.AddModelFromFile("models/ball.sdf")
# floor = parser.AddModelFromFile("models/floor.sdf")
# paddle = parser.AddModelFromFile("models/paddle.sdf")
# plant.WeldFrames(
#     plant.GetFrameByName("iiwa_link_7"), 
#     plant.GetFrameByName("base_link"), 
#     RigidTransform(RollPitchYaw(0, -np.pi/2, 0), [0, 0, 0.25]))

plant.gravity_field().set_gravity_vector([0., 0., -0.1])    

plant.Finalize()

visualizer = MeshcatVisualizerCpp.AddToBuilder(
    builder, 
    scene_graph, 
    meshcat,
    MeshcatVisualizerParams(delete_prefix_initialization_event=False))

In [44]:
controller_plant = MultibodyPlant(time_step = 1e-2)
controller_iiwa = AddIiwa(controller_plant)
controller_parser = Parser(controller_plant)
# controller_parser.AddModelFromFile("models/paddle.sdf")
# controller_plant.WeldFrames(
#     controller_plant.GetFrameByName("iiwa_link_7"),
#     controller_plant.GetFrameByName("base_link"),
#     RigidTransform(RollPitchYaw(0, -np.pi/2, 0), [0, 0, 0.25]))
controller_plant.Finalize()

Kp = np.full(7, 100)
Ki = 2 * np.sqrt(Kp)
Kd = np.full(7, 1)
iiwa_controller = builder.AddSystem(InverseDynamicsController(controller_plant, Kp, Ki, Kd, False))
iiwa_controller.set_name("iiwa_controller");
builder.Connect(plant.get_state_output_port(iiwa),
                iiwa_controller.get_input_port_estimated_state())
builder.Connect(iiwa_controller.get_output_port_control(),
                plant.get_actuation_input_port())

In [45]:
diagram = builder.Build()
context = diagram.CreateDefaultContext()
diagram.Publish(context)

In [46]:
q0 = np.array([-1.57, 0.1, 0, -1.2, 0, 1.6, 0])
x0 = np.hstack((q0, 0*q0))

plant_context = plant.GetMyMutableContextFromRoot(context)

plant.SetPositions(plant_context, iiwa, q0)
# plant.SetPositions(plant_context, ball, np.array([1, 1, 1, 1, 1, 0, 5]))
iiwa_controller.GetInputPort('desired_state').FixValue(iiwa_controller.GetMyMutableContextFromRoot(context), x0)

In [47]:
simulator = Simulator(diagram, context)
simulator.set_target_realtime_rate(0.25)
simulator.AdvanceTo(1.0)